In [ ]:
!pip install openai langchain faiss-cpu
!pip install langchain
!pip install openai
!pip install python-magic
!pip install chromadb
!pip install unstructured
!pip install tiktoken


In [70]:
import os
import nltk
from google.colab import files
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from nltk.tokenize import word_tokenize
from openai.error import InvalidRequestError

In [ ]:
os.environ['OPENAI_API_KEY'] = '...'

nltk.download('punkt')

nltk.download('averaged_perceptron_tagger')


In [72]:
class Document:
    def __init__(self, text):
        self.page_content = text
        self.metadata = {}



In [73]:
uploaded_files = files.upload()



Saving test2-Sheet1.txt to test2-Sheet1 (16).txt


In [74]:
documents = []
for fn in uploaded_files.keys():
    # Check if the uploaded file is too large to process in one go
    if len(uploaded_files[fn]) > 1e6:  # 1e6 bytes == 1MB
        print(f"File {fn} is too large to process in one go. It will be processed in chunks.")
        # Process the file in chunks
        for i in range(0, len(uploaded_files[fn]), int(1e6)):  # Process chunks of 1MB
            document_content = uploaded_files[fn][i:i+int(1e6)].decode("utf-8")  # Convert bytes to string
            documents.append(Document(document_content))
    else:
        document_content = uploaded_files[fn].decode("utf-8")  # Convert bytes to string
        documents.append(Document(document_content))

In [75]:
# Split the documents into manageable chunks
text_splitter = CharacterTextSplitter(chunk_size=2048, chunk_overlap=0) 

In [76]:
# Tokenize and split documents
documents_tokenized = []
for doc in documents:
    tokens = word_tokenize(doc.page_content)
    chunks = [' '.join(tokens[i:i+2048]) for i in range(0, len(tokens), 2048)]
    documents_tokenized.extend([Document(chunk) for chunk in chunks])

texts = text_splitter.split_documents(documents_tokenized)

In [81]:
# Generate embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

# Generate Chroma vectorstore from documents
docsearch = Chroma.from_documents(texts, embeddings)

# Create VectorDBQA instance
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)

# Run a query on each document separately
query = "create curations of publications about eCommerce"

In [85]:
responses = []  # Store the responses from each document

for document in documents_tokenized:
    docsearch = Chroma.from_documents([document], embeddings)  # Process only one document at a time
    qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)
    try:
        response = qa.run(query)
        responses.append(response)
    except InvalidRequestError as e:
        print(f"Query was too large for document: {document}. Skipping...")
# print all responses
for response in responses:
    print(response)

 I'm not sure.
 I don't know.


I don't know how to create curations of publications about eCommerce, but some of the publications listed that are related to eCommerce include: 
- Xiao & Benbasat (2011), "Product-Related Deception in E-Commerce: A Theoretical Perspective", MIS Quarterly, 35(1), 169. 
- Ou, Pavlou, & Davison (2014), "Swift Guanxi in Online Marketplaces: The Role of Computer-Mediated Communication Technologies", MIS Quarterly, 38(1), 209-230.
- Tanriverdi (2006), "Performance Effects of Information Technology Synergies in Multibusiness Firms", MIS Quarterly, 30(1), 57.
- Hui, Teo, & Lee (2007), "The Value of Privacy Assurance: An Exploratory Field Experiment", MIS Quarterly, 31(1), 19.
 This question is too broad to answer with the given context.
 I don't know.
